In [1]:
# import nedded libraries
import pandas as pd
import numpy as np
#Data visualisation libraries 
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
from sklearn.model_selection import train_test_split,learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# data describtion
df = pd.read_csv(r'C:\Users\Amr.Abdelfattah\machine-learning-course\ALT1088.csv')
#df.drop(0, axis=0, inplace=True)
display(df.head())
display(df.info())
display(df.describe())
display(df.columns)

In [ ]:
sns.distplot(df['VCL_GEO_QEPP'])
sns.distplot(df['PHIT_QEPP'])


In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.heatmap(df.corr(),annot=True,ax=ax,linewidths=0.1)

In [ ]:
df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
df_new = df.drop(columns = ['wellName', 'datasetName', 'RXOZ','TDEP','SLOANI', 'HCGR', 'DTSH_SLOW', 'DTSH_FAST',
                            'FWID_DF','AT60', 'AT90','PEFZ','PIGE_QEPP','MINXENE_OVERALL', 'GR_EDTC','DTST_STM'],axis=1)
df_new

In [ ]:
df_new.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.heatmap(df_new.corr(),annot=True,ax=ax,linewidths=0.1)

In [ ]:
df_new.corr()

In [ ]:
df_new.replace(-9999.000000, np.nan, inplace=True)
display(df_new.isna().sum())
df_new.dropna(inplace=True)
df_new

In [ ]:
df_new.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.heatmap(df_new.corr(),annot=True,ax=ax,linewidths=0.1)

In [ ]:
df_new = df_new.drop(columns = ['AT20','AT30'],axis=1)
fig, ax = plt.subplots(figsize=(12,8))
sns.heatmap(df_new.corr(),annot=True,ax=ax,linewidths=0.1)

In [ ]:
df_new


In [ ]:
# feature scaling and model training
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score,train_test_split
min_max_scaler = preprocessing.MinMaxScaler()
X = df_new.loc[:,df_new.columns!=' 	Fracture Density'] # features
Y = df_new['Fracture Density'] # label
X = min_max_scaler.fit_transform(X)
linear_reg = LinearRegression()
xtrain,xtest,ytrain,ytest=train_test_split(X,Y,test_size=0.2)
linear_reg.fit(xtrain,ytrain)
preds=linear_reg.predict(xtest)
print('model rmse = ',np.mean((preds-ytest)**2))

In [ ]:
# test model with cross validation
# visualize rmse scores
import plotly.graph_objects as go
scores = cross_val_score(linear_reg, X , Y, scoring="neg_mean_squared_error", cv=10)
rmse_scores = -scores
fig = go.Figure(
    data=[go.Bar(x=[i for i in range(10)],y=list(rmse_scores)[:10])],
    layout_title_text="cv versus model error")
fig.show()

In [ ]:
lm = LinearRegression()
train_sizes, train_scores, test_scores = learning_curve(estimator=lm,
                                                        X=xtrain,
                                                        y=ytrain,
                                                        scoring = 'neg_mean_squared_error',
                                                       cv=10)
lm.fit(xtrain,ytrain)

In [ ]:
predictions = lm.predict(xtest)
error=np.sqrt(mean_squared_error(ytest,predictions)) # rmse
print("RMSE : ",error)

In [ ]:
print("intercept : ",lm.intercept_)
for idx, col_name in enumerate(xtrain.columns):
    print("The coefficient for {} is {}".format(col_name, lm.coef_[idx]))

In [ ]:
train_mean = -np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Create means and standard deviations of test set scores
test_mean = -np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Draw lines
plt.plot(train_sizes, train_mean, '--', color="#111111",  label="Training score")
plt.plot(train_sizes, test_mean, color="#111111", label="Cross-validation score")

# Draw bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="#DDDDDD")
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="#DDDDDD")

# Create plot
plt.title("Learning Curve")
plt.xlabel("Training Set Size"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# check model fitted lines against different features
plt.figure(figsize=(10,6))

plt.subplot(2, 2, 1)
plt.scatter(df['DTCO_Spline'],df['Fracture Density'])
plt.plot(df['DTCO_Spline'],'r')
plt.xlabel('DTCO_Spline')
plt.ylabel('Fracture Density')

plt.subplot(2, 2, 2)
plt.scatter(df['AT10'],df['Fracture Density'])
plt.plot(df['AT10'],'r')
plt.xlabel('AT10')
plt.ylabel('Fracture Density')

plt.subplot(2, 2, 3)
plt.scatter(df['PHIT_QEPP'],df['Fracture Density'])
plt.plot(df['PHIT_QEPP'],'r')
plt.xlabel('PHIT_QEPP')
plt.ylabel('Fracture Density')

plt.show()